In [78]:
#Load the libraries
import pandas as pd
import re

# For the 3 task. I create a simple naive bayes classifier to predict if the SMS is ham or spam. 

# Load the "spam-ham" data

In [79]:
df = pd.read_csv('spam.csv')
df.rename(columns={'v1':'label', 'v2':'SMS'}, inplace=True)
df = df[['SMS', 'label']]

In [80]:
df.head(5)

,SMS,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [81]:
df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [82]:
df.sample(frac=1).reset_index(inplace=True)

# Train and Test split

In [83]:
# Calculate index for split
training_test_index = round(len(df) * 0.8)

# Split into training and test sets
training_set = df[:training_test_index].reset_index(drop=True)
test_set = df[training_test_index:].reset_index(drop=True)

In [84]:
# Remove punctuation and set the text to lower case

training_set['SMS'] = training_set['SMS'].str.replace(
   '\W', ' ') # Removes punctuation
training_set['SMS'] = training_set['SMS'].str.lower()

<ipython-input-84-d007576629cc>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set['SMS'] = training_set['SMS'].str.replace(


In [85]:
# Create the vocabulary list

# Split the text into chunks of words
training_set['SMS'] = training_set['SMS'].str.split()

# Create a list with all de vocabulary
vocabulary = []
for SMS in training_set['SMS']:
    for word in SMS:
        vocabulary.append(word)
# set to unique words
vocabulary = list(set(vocabulary))

In [86]:
len(vocabulary)

7750

In [87]:
# Create a table with rows as SMS and columns as words. So there will be a 1 in each word that the SMS includes.

word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [ ]:
# So for example the word "looks" only appears in one SMS, so for all the other SMS it will have a 0

In [88]:
word_counts_per_sms['looks']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [40]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,flea,that2worzels,gt,hand,grand,listen,serena,5,75max,44,...,eh,damn,greetings,box177,0089,382,emc1,mmmmm,eating,looks
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Concat with the original DF

In [41]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,SMS,label,flea,that2worzels,gt,hand,grand,listen,serena,5,...,eh,damn,greetings,box177,0089,382,emc1,mmmmm,eating,looks
0,"[go, until, jurong, point, crazy, available, o...",ham,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[ok, lar, joking, wif, u, oni]",ham,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[free, entry, in, 2, a, wkly, comp, to, win, f...",spam,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[u, dun, say, so, early, hor, u, c, already, t...",ham,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[nah, i, don, t, think, he, goes, to, usf, he,...",ham,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam. ¿How many words are in spam SMS?
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

In [44]:
# Initiate parameters. # Here we will calculate the probability for each word in the ham and spam space.
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters (prob)
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam

    n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

In [46]:
def classify(message):
''' We insert a message as parameter and return the predicted label'''

    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    # Set the prior probs
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
        # Multiply each word probabilities
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham: 
            p_ham_given_message *= parameters_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [47]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.2397944750368253e-25
P(Ham|message): 2.586260759360702e-27
Label: Spam


In [90]:
classify("Sounds good, Juan, see u there")

P(Spam|message): 7.5115536979812e-22
P(Ham|message): 1.5164009496788251e-18
Label: Ham


In [49]:
def classify_test_set(message):
''' Similar function but in this case we use it in the dataframe'''

    message = re.sub('\W', ' ', message)
    message = message.lower().split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [62]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,SMS,label,predicted
0,Welcome to UK-mobile-date this msg is FREE giv...,spam,spam
1,This is wishing you a great day. Moji told me ...,ham,ham
2,Thanks again for your reply today. When is ur ...,ham,ham
3,"Sorry I flaked last night, shit's seriously go...",ham,ham
4,He said i look pretty wif long hair wat. But i...,ham,ham


In [63]:
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
    row = row[1]
    if row['label'] == row['predicted']:
        correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1098
Incorrect: 16
Accuracy: 0.9856373429084381
